In [1]:
import pandas as pd

damages_df = pd.read_csv('Damages.csv', index_col=0)
player_frames_df = pd.read_csv('PlayerFrames.csv', index_col=0)
bomb_events_df = pd.read_csv('BombEvents.csv', index_col=0)
rounds_df = pd.read_csv('Rounds.csv', index_col=0)
kills_df = pd.read_csv('Kills.csv', index_col=0)


In [2]:
game_state_cols = ['MatchId', 'MapName', 'RoundNum', 'TicksSinceStart', 'CTStartEqVal', 'TStartEqVal', 'CTAlive', 'TAlive', 'CTTotalHP', 'TTotalHP', 'BombPlantedA', 'BombPlantedB', 'CTClosestA', 'TClosestA', 'CTClosestB', 'TClosestB']

## Remove events before and after the round

In [3]:
# TODO: Check that only the post-round time is getting cut (weird since the endtick and officialendtick are practically the same)

In [4]:
m_bomb_events_df = bomb_events_df.merge(rounds_df[['MatchId', 'MapName', 'RoundNum', 'FreezeTimeEnd', 'EndTick']], how='left', on=['MatchId', 'MapName', 'RoundNum'], validate='many_to_one')
m_bomb_events_df[(m_bomb_events_df.Tick <= m_bomb_events_df.FreezeTimeEnd) | (m_bomb_events_df.Tick > m_bomb_events_df.EndTick)]

,MatchId,MapName,RoundNum,Tick,Second,PlayerSteamId,PlayerName,PlayerTeam,PlayerX,PlayerY,PlayerZ,BombAction,BombSite,FreezeTimeEnd,EndTick
21,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_nuke,3,44409,112.289062,76561198051070311,★ ㉑ phx,SWS Gaming,614.699280,-769.868225,-415.968750,plant,A,30036,44232
44,Flashpoint-3-ClosedQualifier-complexity-vs-mou...,de_overpass,24,355110,113.086614,76561198068422762,frozen,mousesports,-2508.249512,780.858887,475.031250,plant,A,356299,356299
100,DreamHack-Masters-Spring-2021-g2-vs-mousesport...,de_vertigo,26,472638,92.046875,76561198012872053,huNter-,G2 Esports,-2216.002686,861.404846,11744.031250,plant,B,475441,475441
141,DreamHack-Masters-Spring-2021-big-vs-heroic-bo3,de_mirage,26,418197,104.515625,76561198160709585,sjuush,HEROIC,-1890.379028,277.885101,-159.968750,plant,B,419944,419944
146,Flashpoint-3-ClosedQualifier-dbl-poney-vs-havu...,de_train,7,128254,44.787402,76561198002132867,ZOREE,HAVU GAMING,723.178406,140.948746,-215.412201,plant,A,122566,127622
159,Flashpoint-3-ClosedQualifier-dbl-poney-vs-havu...,de_train,27,426506,111.330709,76561197960710573,NBK,DBL PONEY,348.260284,8.390979,-215.968750,plant,A,428647,428647
200,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,17,288782,89.740157,76561198077802866,poizon,Complexity,1896.010376,620.768494,160.031250,plant,A,277385,288299
240,DreamHack-Masters-Spring-2021-spirit-vs-virtus...,de_inferno,16,291727,117.000000,76561198040577200,sdy,Team Spirit,1879.965942,318.517883,160.031250,plant,A,276751,291471
253,DreamHack-Masters-Spring-2021-spirit-vs-virtus...,de_train,16,284922,77.385827,76561198127236290,qikert,Virtus.pro,-234.087555,-1281.694336,-355.968750,plant,B,275094,284904
260,DreamHack-Masters-Spring-2021-spirit-vs-virtus...,de_train,27,487455,57.031496,76561198036125584,Jame,Virtus.pro,-166.624283,-1282.281982,-281.276611,plant,B,492196,492196


In [5]:
def keep_in_round(df):
    df = df.merge(rounds_df[['MatchId', 'MapName', 'RoundNum', 'FreezeTimeEnd', 'EndTick']], how='left', on=['MatchId', 'MapName', 'RoundNum'], validate='many_to_one')
    df = df[(df.Tick >= df.FreezeTimeEnd) & (df.Tick < df.EndTick)]
    df.drop(['FreezeTimeEnd', 'EndTick'], axis=1)
    return df

damages_df = keep_in_round(damages_df)
player_frames_df = keep_in_round(player_frames_df)
bomb_events_df = keep_in_round(bomb_events_df)
kills_df = keep_in_round(kills_df)




## Bomb Plants

In [6]:
bomb_events_df.describe()

,RoundNum,Tick,Second,PlayerSteamId,PlayerX,PlayerY,PlayerZ,FreezeTimeEnd,EndTick
count,302.000000,302.000000,302.000000,3.020000e+02,302.000000,302.000000,302.000000,302.000000,302.000000
mean,14.920530,244260.314570,73.359151,7.656120e+16,-127.250626,378.941231,1519.268287,234906.576159,248012.062914
std,9.080316,153453.350869,23.902005,7.512620e+07,1308.562627,1513.599622,4018.216817,153327.133259,153287.729092
min,1.000000,4138.000000,23.031250,7.656120e+16,-2585.803955,-2209.088135,-771.968750,0.000000,4556.000000
25%,7.000000,117919.250000,53.191406,7.656120e+16,-1166.488892,-692.899689,-179.062012,110513.000000,121772.750000
50%,14.000000,219904.000000,73.564930,7.656120e+16,-165.031250,215.153748,98.031250,212681.000000,221513.000000
75%,22.000000,354712.500000,93.199342,7.656120e+16,654.587173,873.797882,160.031250,347592.000000,359566.750000
max,40.000000,664818.000000,115.165354,7.656120e+16,2130.126953,3020.997070,11776.031250,657459.000000,667569.000000


In [7]:
bomb_plants_df = bomb_events_df[bomb_events_df.BombAction == 'plant'].drop(columns='BombAction')

In [8]:
bomb_plants_df.describe()

,RoundNum,Tick,Second,PlayerSteamId,PlayerX,PlayerY,PlayerZ,FreezeTimeEnd,EndTick
count,302.000000,302.000000,302.000000,3.020000e+02,302.000000,302.000000,302.000000,302.000000,302.000000
mean,14.920530,244260.314570,73.359151,7.656120e+16,-127.250626,378.941231,1519.268287,234906.576159,248012.062914
std,9.080316,153453.350869,23.902005,7.512620e+07,1308.562627,1513.599622,4018.216817,153327.133259,153287.729092
min,1.000000,4138.000000,23.031250,7.656120e+16,-2585.803955,-2209.088135,-771.968750,0.000000,4556.000000
25%,7.000000,117919.250000,53.191406,7.656120e+16,-1166.488892,-692.899689,-179.062012,110513.000000,121772.750000
50%,14.000000,219904.000000,73.564930,7.656120e+16,-165.031250,215.153748,98.031250,212681.000000,221513.000000
75%,22.000000,354712.500000,93.199342,7.656120e+16,654.587173,873.797882,160.031250,347592.000000,359566.750000
max,40.000000,664818.000000,115.165354,7.656120e+16,2130.126953,3020.997070,11776.031250,657459.000000,667569.000000


In [9]:
bomb_plants_df.isna().sum()

MatchId          0
MapName          0
RoundNum         0
Tick             0
Second           0
PlayerSteamId    0
PlayerName       0
PlayerTeam       0
PlayerX          0
PlayerY          0
PlayerZ          0
BombSite         0
FreezeTimeEnd    0
EndTick          0
dtype: int64

In [10]:
bomb_plants_df['BombPlantedA'] = bomb_plants_df['BombSite'] == 'A'
bomb_plants_df['BombPlantedB'] = bomb_plants_df['BombSite'] == 'B'
bomb_plants_df.drop(columns=['Second', 'PlayerSteamId', 'PlayerName', 'PlayerTeam', 'PlayerX', 'PlayerY', 'PlayerZ', 'BombSite'], inplace=True)
bomb_plants_df.head()

,MatchId,MapName,RoundNum,Tick,FreezeTimeEnd,EndTick,BombPlantedA,BombPlantedB
0,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,23419,15794,27867,True,False
2,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,2,41686,31070,46939,False,True
3,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,3,59274,50138,64531,False,True
4,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,4,75832,67724,81080,False,True
5,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,5,94851,84283,98592,False,True


## Kills and Damage

In [11]:
# Drop C4 kills and damages since those happen when the round ends
kills_df = kills_df[kills_df.Weapon != 'C4']
damages_df = damages_df[damages_df.Weapon != 'C4']

In [12]:
kills_df.drop(columns=['Second', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId', 'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'VictimX', 'VictimY', 'VictimZ', 'VictimAreaId', 'VictimAreaName', 'VictimViewX', 'VictimViewY', 'AssisterSteamId', 'AssisterName', 'AssisterTeam', 'AssisterSide', 'AssisterX', 'AssisterY', 'AssisterZ', 'AssisterAreaId', 'AssisterAreaName', 'IsWallbang', 'IsFlashed', 'IsHeadshot','IsTrade', 'PlayerTradedName', 'PlayerTradedTeam', 'PlayerTradedSteamId', 'IsFirstKill'], inplace=True)
kills_df.head()

,MatchId,MapName,RoundNum,Tick,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,VictimSteamId,VictimName,VictimTeam,VictimSide,Weapon,FreezeTimeEnd,EndTick
0,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,20967,7.656120e+16,★ ㉑ cass1n,Santos e-Sports,T,76561198286138224,★ ㉑ BGalvao90,SWS Gaming,CT,p250,15794,27867
1,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,21115,7.656120e+16,★ ㉑ cass1n,Santos e-Sports,T,76561198135228658,★ ㉑ gafolo,SWS Gaming,CT,p250,15794,27867
2,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,21259,7.656120e+16,★ ㉑ matios,SWS Gaming,CT,76561198148561509,★ ㉑ cass1n,Santos e-Sports,T,USP-S,15794,27867
3,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,21799,7.656120e+16,★ ⑳ MaLLby,Santos e-Sports,T,76561198061789000,★ ㉑ RICIOLI,SWS Gaming,CT,Glock-18,15794,27867
4,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,22791,7.656120e+16,★ ⑳ MaLLby,Santos e-Sports,T,76561198051070311,★ ㉑ phx,SWS Gaming,CT,Glock-18,15794,27867


In [13]:
damages_df.drop(columns=['Second', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId', 'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'VictimX', 'VictimY', 'VictimZ', 'VictimAreaId', 'VictimAreaName', 'VictimViewX', 'VictimViewY', 'HpDamage', 'ArmorDamage', 'ArmorDamageTaken', 'HitGroup'], inplace=True)
damages_df.head()

,MatchId,MapName,RoundNum,Tick,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,VictimSteamId,VictimName,VictimTeam,VictimSide,Weapon,HpDamageTaken,FreezeTimeEnd,EndTick
0,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,20943,7.656120e+16,★ ㉑ cass1n,Santos e-Sports,T,76561198286138224,★ ㉑ BGalvao90,SWS Gaming,CT,p250,21,15794,27867
1,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,20967,7.656120e+16,★ ㉑ cass1n,Santos e-Sports,T,76561198286138224,★ ㉑ BGalvao90,SWS Gaming,CT,p250,79,15794,27867
2,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,21115,7.656120e+16,★ ㉑ cass1n,Santos e-Sports,T,76561198135228658,★ ㉑ gafolo,SWS Gaming,CT,p250,100,15794,27867
3,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,21259,7.656120e+16,★ ㉑ matios,SWS Gaming,CT,76561198148561509,★ ㉑ cass1n,Santos e-Sports,T,USP-S,100,15794,27867
4,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,21759,7.656120e+16,★ ⑳ MaLLby,Santos e-Sports,T,76561198061789000,★ ㉑ RICIOLI,SWS Gaming,CT,Glock-18,11,15794,27867


In [14]:
# Check that all kills have a corresponding damage event
inner_merge_df = kills_df.merge(damages_df, on=['MatchId', 'MapName', 'RoundNum', 'Tick', 'AttackerSteamId', 'AttackerName', 'AttackerTeam', 'AttackerSide', 'VictimSteamId', 'VictimName', 'VictimTeam', 'VictimSide'])
print('Number of kills with corresponding damage event:', len(inner_merge_df))
print('Number of kills:', len(kills_df))

Number of kills with corresponding damage event: 3325
Number of kills: 3317


In [15]:
# Combine simultaneous instances of damage from the same player to the same player
damages_df = damages_df.groupby(['MatchId', 'MapName', 'RoundNum', 'Tick', 'AttackerSteamId', 'AttackerName', 'AttackerTeam', 'AttackerSide', 'VictimSteamId', 'VictimName', 'VictimTeam', 'VictimSide'], as_index=False).sum()
damages_df['HpDamageTaken'] = damages_df.HpDamageTaken.apply(lambda x: min([x, 100]))  # Bugged/laggy damages instances > 100 (shouldn't be possible)


In [16]:
# Check that all kills NOW have a corresponding damage event
merge_df = kills_df.merge(damages_df, how='left', on=['MatchId', 'MapName', 'RoundNum', 'Tick', 'AttackerSteamId', 'AttackerName', 'AttackerTeam', 'AttackerSide', 'VictimSteamId', 'VictimName', 'VictimTeam', 'VictimSide'], indicator=True)
merge_df[merge_df._merge == 'left_only']

,MatchId,MapName,RoundNum,Tick,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,VictimSteamId,VictimName,VictimTeam,VictimSide,Weapon,FreezeTimeEnd_x,EndTick_x,HpDamageTaken,FreezeTimeEnd_y,EndTick_y,_merge


In [17]:
inner_merge_df = kills_df.merge(damages_df, on=['MatchId', 'MapName', 'RoundNum', 'Tick', 'AttackerSteamId', 'AttackerName', 'AttackerTeam', 'AttackerSide', 'VictimSteamId', 'VictimName', 'VictimTeam', 'VictimSide'])

### Merging kills into damage

In [18]:
kills_df['IsKill'] = True
damages_df = pd.merge(damages_df, kills_df, how='left', on=['MatchId', 'MapName', 'RoundNum', 'Tick', 'AttackerSteamId', 'AttackerName', 'AttackerTeam', 'AttackerSide', 'VictimSteamId', 'VictimName', 'VictimTeam', 'VictimSide'], validate='one_to_one')
damages_df['IsKill'].fillna(False, inplace=True)
damages_df.head()

,MatchId,MapName,RoundNum,Tick,AttackerSteamId,AttackerName,AttackerTeam,AttackerSide,VictimSteamId,VictimName,VictimTeam,VictimSide,HpDamageTaken,FreezeTimeEnd_x,EndTick_x,Weapon,FreezeTimeEnd_y,EndTick_y,IsKill
0,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,6431,7.656120e+16,RUSH,Complexity,CT,76561198131369187,Bubzkji,Astralis,T,16,3710,12332,NaN,NaN,NaN,False
1,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,6499,7.656120e+16,RUSH,Complexity,CT,76561198131369187,Bubzkji,Astralis,T,16,3710,12332,NaN,NaN,NaN,False
2,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,6522,7.656120e+16,RUSH,Complexity,CT,76561198131369187,Bubzkji,Astralis,T,20,3710,12332,NaN,NaN,NaN,False
3,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,6543,7.656120e+16,blameF,Complexity,CT,76561198131369187,Bubzkji,Astralis,T,48,3710,12332,USP-S,3710.0,12332.0,True
4,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,6646,7.656120e+16,dupreeh,Astralis,T,76561198023592320,RUSH,Complexity,CT,12,3710,12332,NaN,NaN,NaN,False


In [19]:
damages_groups = damages_df.groupby(['MatchId', 'MapName', 'RoundNum', 'VictimSide'])
damages_groups.groups

{('DreamHack-Masters-Spring-2021-astralis-vs-complexity-bo3', 'de_inferno', 1, 'CT'): [4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 17, 18, 19, 21, 22, 24, 27], ('DreamHack-Masters-Spring-2021-astralis-vs-complexity-bo3', 'de_inferno', 1, 'T'): [0, 1, 2, 3, 6, 9, 16, 20, 23, 25, 26, 28], ('DreamHack-Masters-Spring-2021-astralis-vs-complexity-bo3', 'de_inferno', 2, 'CT'): [29, 33, 34, 35, 36, 41, 42, 43, 45, 46, 47, 48, 49, 51], ('DreamHack-Masters-Spring-2021-astralis-vs-complexity-bo3', 'de_inferno', 2, 'T'): [30, 31, 32, 37, 38, 39, 40, 44, 50], ('DreamHack-Masters-Spring-2021-astralis-vs-complexity-bo3', 'de_inferno', 3, 'CT'): [52, 53, 55, 63, 65], ('DreamHack-Masters-Spring-2021-astralis-vs-complexity-bo3', 'de_inferno', 3, 'T'): [54, 56, 57, 58, 59, 60, 61, 62, 64, 66, 67, 68, 69, 70, 71], ('DreamHack-Masters-Spring-2021-astralis-vs-complexity-bo3', 'de_inferno', 4, 'CT'): [79, 81, 85, 87, 88, 89, 90, 91, 93, 102, 103, 107, 108], ('DreamHack-Masters-Spring-2021-astralis-vs-complexity-bo3'

## Player Movements

In [20]:
def count_groups(df):
    df[df['VictimSide'] + 'Alive'] = 5 - df['IsKill'].cumsum()
    return df

damages_groups = damages_df.groupby(['MatchId', 'MapName', 'RoundNum', 'VictimSide']).apply(count_groups)
damages_groups.describe()

,RoundNum,Tick,AttackerSteamId,VictimSteamId,HpDamageTaken,FreezeTimeEnd_x,EndTick_x,FreezeTimeEnd_y,EndTick_y,CTAlive,TAlive
count,13296.000000,13296.000000,1.329600e+04,1.329600e+04,13296.000000,1.329600e+04,1.329600e+04,3317.000000,3317.000000,6086.000000,7210.000000
mean,14.661928,235949.528054,7.656120e+16,7.656120e+16,29.171104,2.300866e+05,2.417781e+05,224607.781730,236319.182092,1.715577,1.385714
std,8.756852,147863.847012,9.670330e+07,1.070778e+08,26.430019,1.509526e+05,1.513383e+05,146806.371279,147046.695634,1.059035,1.184840
min,1.000000,173.000000,7.656120e+16,7.656120e+16,1.000000,0.000000e+00,1.150000e+03,0.000000,1150.000000,1.000000,0.000000
25%,7.000000,115943.000000,7.656120e+16,7.656120e+16,11.000000,1.087850e+05,1.200650e+05,103631.000000,117188.000000,1.000000,1.000000
50%,14.000000,224537.500000,7.656120e+16,7.656120e+16,22.000000,2.197230e+05,2.285200e+05,208745.000000,219781.000000,1.000000,1.000000
75%,21.000000,332894.000000,7.656120e+16,7.656120e+16,33.000000,3.268920e+05,3.375480e+05,322306.000000,335980.000000,2.000000,2.000000
max,41.000000,684422.000000,7.656120e+16,7.656120e+16,100.000000,1.953390e+06,2.032044e+06,670516.000000,684435.000000,5.000000,5.000000


In [21]:
list(player_frames_df)

['MatchId',
 'MapName',
 'RoundNum',
 'Tick',
 'Second',
 'Side',
 'TeamName',
 'PlayerName',
 'PlayerSteamId',
 'X',
 'Y',
 'Z',
 'ViewX',
 'ViewY',
 'AreaId',
 'Hp',
 'Armor',
 'IsAlive',
 'IsFlashed',
 'IsAirborne',
 'IsDucking',
 'IsScoped',
 'IsWalking',
 'EqValue',
 'HasHelmet',
 'HasDefuse',
 'DistToBombsiteA',
 'DistToBombsiteB',
 'FreezeTimeEnd',
 'EndTick']

In [22]:
player_frames_df.drop(columns=['Second', 'X', 'Y', 'Z', 'ViewX', 'ViewY', 'AreaId', 'Hp', 'Armor', 'IsAlive', 'IsFlashed', 'IsAirborne', 'IsDucking', 'IsScoped', 'IsWalking', 'EqValue', 'HasHelmet', 'HasDefuse', 'PlayerName', 'PlayerSteamId'], inplace=True)
player_frames_df.head()

,MatchId,MapName,RoundNum,Tick,Side,TeamName,DistToBombsiteA,DistToBombsiteB,FreezeTimeEnd,EndTick
0,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,16046,CT,SWS Gaming,28,36,15794,27867
1,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,16046,CT,SWS Gaming,13,39,15794,27867
2,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,16046,CT,SWS Gaming,9,59,15794,27867
3,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,16046,CT,SWS Gaming,14,40,15794,27867
4,Liga-Gamers-Club-2021-Serie-A-April-Cup-santos...,de_overpass,1,16046,CT,SWS Gaming,13,41,15794,27867


In [23]:
def min_per_side(df):
    CTMinDistA
    CTMinDistB
    TMinDistA
    TMinDistB

site_distances_df = player_frames_df.groupby(['MatchId', 'MapName', 'RoundNum', 'Tick', 'Side', 'TeamName'], as_index=False).aggregate('min')
site_distances_df.head()

,MatchId,MapName,RoundNum,Tick,Side,TeamName,DistToBombsiteA,DistToBombsiteB,FreezeTimeEnd,EndTick
0,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3740,CT,Complexity,19,15,3710,12332
1,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3740,T,Astralis,51,42,3710,12332
2,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3869,CT,Complexity,17,14,3710,12332
3,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3869,T,Astralis,46,37,3710,12332
4,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3998,CT,Complexity,10,12,3710,12332


def combine_sides(df):


site_distances_df.groupby(['MatchId', 'MapName', 'RoundNum', 'Tick']).apply()

## Putting it all together

In [24]:
bomb_plants_df['EventType'] = 'BombPlant'
damages_df['EventType'] = 'DamageDealt'
site_distances_df['EventType'] = 'PositionUpdate'

In [25]:
game_states_df = pd.concat([bomb_plants_df, damages_df, site_distances_df], axis=0, ignore_index=True)
game_states_df.sort_values(['MatchId', 'MapName', 'RoundNum', 'Tick'], ignore_index=True, inplace=True)
game_states_df[game_states_df['EventType'] == 'PositionUpdate'].head(20)

,MatchId,MapName,RoundNum,Tick,FreezeTimeEnd,EndTick,BombPlantedA,BombPlantedB,EventType,AttackerSteamId,...,FreezeTimeEnd_x,EndTick_x,Weapon,FreezeTimeEnd_y,EndTick_y,IsKill,Side,TeamName,DistToBombsiteA,DistToBombsiteB
0,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3740,3710.0,12332.0,NaN,NaN,PositionUpdate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CT,Complexity,19.0,15.0
1,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3740,3710.0,12332.0,NaN,NaN,PositionUpdate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,T,Astralis,51.0,42.0
2,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3869,3710.0,12332.0,NaN,NaN,PositionUpdate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CT,Complexity,17.0,14.0
3,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3869,3710.0,12332.0,NaN,NaN,PositionUpdate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,T,Astralis,46.0,37.0
4,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3998,3710.0,12332.0,NaN,NaN,PositionUpdate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CT,Complexity,10.0,12.0
5,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,3998,3710.0,12332.0,NaN,NaN,PositionUpdate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,T,Astralis,43.0,34.0
6,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,4127,3710.0,12332.0,NaN,NaN,PositionUpdate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CT,Complexity,8.0,17.0
7,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,4127,3710.0,12332.0,NaN,NaN,PositionUpdate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,T,Astralis,39.0,30.0
8,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,4256,3710.0,12332.0,NaN,NaN,PositionUpdate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CT,Complexity,6.0,13.0
9,DreamHack-Masters-Spring-2021-astralis-vs-comp...,de_inferno,1,4256,3710.0,12332.0,NaN,NaN,PositionUpdate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,T,Astralis,37.0,28.0
